## Q2 Implementing Bayesian Filter and using it to predict the weather

In [1]:
import numpy as np
from numpy import linalg as LA
from sklearn.preprocessing import normalize
import random
from math import log
import scipy

In [36]:
# Transition matrix
sunny = np.array([0.8, 0.2, 0])
cloudy = np.array([0.4, 0.4, 0.2])
rainy = np.array([0.2, 0.6, 0.2])

transitionMatrix = np.array([sunny,cloudy,rainy])

### b) Implementation

In [37]:
def BayesFilter(priorArray, generativeArray, transitionMatrix):
    belief_predict = np.matmul(transitionMatrix, priorArray)
    unnormalized_update = np.multiply(generativeArray, belief_predict)
    belief_update = np.divide(unnormalized_update, np.sum(unnormalized_update))
    return belief_update

### Simulation 

In [38]:
def BayesSimulate(priors, measurementPrediction, state_transition, n):
    for i in range(n):
        priors = BayesFilter(priors, measurementPrediction, state_transition)
    return priors

### c) Simulate to find the stationary distribution

In [39]:
# Simulate for 100 days
# Regardless of the prior it will converge to the same distribution
# Here is more on stationary distributions: https://brilliant.org/wiki/stationary-distributions/
stationaryDistribution = BayesSimulate(np.array([0.3, 0.4, 0.3]), np.array([1, 1, 1]), transitionMatrix.transpose(), 10)
print(stationaryDistribution)

[0.64259993 0.28586496 0.07153511]


### a) Now let's find the probabilities for a series of day assuming the first day was _sunny_

In [40]:
# probability(x_2 = sunny) = 1.0
# probability(x_2 = cloudy) = ?
BayesFilter(np.array([1, 0, 0]), np.array([1, 1, 1]), transitionMatrix.transpose())

array([0.8, 0.2, 0. ])

In [41]:
# probability(x_3 = cloudy) = ?
BayesFilter(np.array([0.8, 0.2, 0.]), np.array([1, 1, 1]), transitionMatrix.transpose())

array([0.72, 0.24, 0.04])

In [42]:
# probability(x_4 = rainy) = ?
BayesFilter(np.array([0.72, 0.24, 0.04]), np.array([1, 1, 1]), transitionMatrix.transpose())

array([0.68 , 0.264, 0.056])

### e) Entropy of the stationary distribution

In [43]:
def entropy(probabilities):
    entrp = 0
    for p in probabilities:
        assert p >= 0.
        entrp = entrp - p*log(p)
    return entrp

In [44]:
print(entropy(stationaryDistribution))

0.830827826643127


### f) Computing the probability table of yesterday from today

You need to use the bayesian rule to do this.

$p(x|y) = \frac{p(y|x)p(x)}{p(y)}$

$\textrm{Therefore} \\
p(w_{yesterday}|w_{today})=\frac{p(w_{today}|w_{yesterday})p(w_{yesterday})}{p(w_{today})}$

We can consider the stationary distribution for the values of $p(w_{yesterday})$ and $p(w_{today})$.

e.g., $p(sunny_{yesterday} | sunny_{today}) = \frac{0.8x0.64}{0.64}$

#### To get the new rows of the state transition matrix, you can multiply each row with the corresponding stationary probability and element-wise divide the entire row with the stationary probability

In [76]:
# P(X_yesterday | sunny_today)
np.divide(sunny*stationaryDistribution[0], stationaryDistribution)

array([0.8       , 0.44958286, 0.        ])

In [77]:
# P(X_yesterday | cloudy_today)
np.divide(cloudy*stationaryDistribution[1], stationaryDistribution)

array([0.17794273, 0.4       , 0.79922976])

In [79]:
# # P(X_yesterday | rainy_today)
np.divide(rainy*stationaryDistribution[2], stationaryDistribution)

array([0.02226428, 0.15014456, 0.2       ])

### g) Seasons
In a Markov Chain the state only depends on the previous state and nothing else.